The code in this notebook is to webscrape the New york postal code,borough and neighborhood information from wikipedia
and clean the acquired dataset

Importing necessary libraries

In [4]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from bs4 import BeautifulSoup
import lxml
import math

Webscrapping the data and storing it in a pandas dataframe

In [5]:
res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))[0]
df = pd.DataFrame(df)
df.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


Drop all rows where borough is not assigned

In [6]:
df1 = df[df["Borough"]!="Not assigned"]
df1=df1.reset_index(drop=True)
df1.head()


,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


Concatenate the neighborhoods with same postal code and borough using ','.
Note: wikipedia has already concatenated such neighborhoods using ' \'. we shall replace it with ',' to get what is expected.

In [7]:
df2 = df1.groupby(['Postal code','Borough'],as_index=True)['Neighborhood'].apply(', '.join).reset_index()
for i in range(df2.shape[0]):
    df2.iloc[i,2]=df2.iloc[i,2].replace(' /',',')
df2.head()


,Postal code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


If neighborhhod is not assigned then it should be replaced with borough name

In [8]:
for i in range(df2.shape[0]):
    if (df2['Neighborhood'][i]==np.nan):
        df2['Neighborhood'][i]=df2["Borough"][i]
        
df2.head()

,Postal code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Find the number of rows in our dataset.

In [9]:
df2.shape[0]

103